# Data Audit

This script looks at how many missing values each feature has, how many unique values the categorical features have then applies preprocessing steps to remove outliers and drop features with too many missing values.

In [1]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt

In [2]:
# set variables from config file
config_path = os.path.abspath('..')

with open(config_path + '\config-example.json', 'r') as f:
    config = json.load(f)

dataset_audited_fname = config['DEFAULT']['dataset_audited_fname']
dataset_sampled_fname = config['DEFAULT']['dataset_sampled_fname']
processing_path = config['DEFAULT']['processing_path']
dataset_fname_suffix = config['DEFAULT']['dataset_fname_suffix']

In [ ]:
client_data = pd.read_csv(os.path.join(processing_path,dataset_sampled_fname) + dataset_fname_suffix, header = 0, delimiter = ',',)

In [4]:
# Change the timestamp to date format and set it as index
client_data['StartDate'] = pd.to_datetime(client_data['StartDate'], infer_datetime_format=True)
client_data['StatusCreatedDate'] = pd.to_datetime(client_data['StatusCreatedDate'], errors='coerce')



In [5]:
client_data['StartYear'] = client_data['StartDate'].dt.year

In [6]:
client_data['StatusCreatedYear'] = client_data['StatusCreatedDate'].dt.year

In [7]:
# data['StartYear'] = data['StartDate'].dt.year
# data['StartMonth'] = data['StartDate'].dt.month
# data['StatusCreatedYear'] = data['StatusCreatedDate'].dt.year
# data['StatusCreatedMonth'] = data['StatusCreatedDate'].dt.month
# data['StartWeek'] = data['StartDate'].dt.isocalendar().week
# data['StatusCreatedWeek'] = data['StatusCreatedDate'].dt.isocalendar().week

In [8]:
client_data['StartYear'].value_counts(normalize=True)

2019    0.285487
2018    0.284479
2022    0.170003
2020    0.136142
2021    0.103659
2017    0.020202
2023    0.000027
Name: StartYear, dtype: float64

In [9]:
client_data['StatusCreatedYear'].value_counts(normalize=True)

2019.0    0.287286
2018.0    0.275106
2022.0    0.160624
2020.0    0.138153
2021.0    0.113048
2017.0    0.025764
2023.0    0.000018
Name: StatusCreatedYear, dtype: float64

In [10]:
print(len(client_data))

658446


In [11]:
client_data.head()

,EventName,StartDate,EventType,BookingReference,AttendeeReference,GroupSize,IsLeadAttendee,AttendeeGrossCost,EventId,BookingStatus,AttendeeType,TicketType,StatusCreatedDate,ClientId,StartYear,StatusCreatedYear
0,Developing Reading and Writing Skills,2017-11-28,"Others, Group 1",B462687,A514297,1,True,0.0,14983,Attending,Attendee,NaN,2017-11-11 09:15:00,153.0,2017,2017.0
1,Teaching Grammar: Classroom choices,2017-12-13,"Others, Group 1",B462689,A514299,1,True,0.0,14960,Attending,Attendee,NaN,2017-11-11 09:18:00,153.0,2017,2017.0
2,Teaching Grammar: Classroom choices,2017-12-13,"Others, Group 1",B467150,A519135,1,True,0.0,14960,Attending,Attendee,NaN,2017-11-22 17:02:00,153.0,2017,2017.0
3,Academic vocabulary: what do students need to ...,2018-04-19,"Others, Group 2",B533234,A588469,1,True,0.0,15893,Attending,Attendee,NaN,2018-11-04 13:38:00,153.0,2018,2018.0
4,Researching the classroom,2018-11-14,"Others, Group 4",B604614,A662691,1,True,0.0,17350,Attending,Attendee,NaN,2018-10-15 14:33:00,153.0,2018,2018.0


### Investigate categoric and numeric fields

In [12]:
# Get numeric and categorical variable names
var_list_num = client_data.select_dtypes(include= 'number').columns.tolist()
var_list_num.remove('ClientId')

var_list_cat = client_data.select_dtypes(include = ['object', 'category']).columns.tolist()

In [13]:
# Percentage of missing values in numeric fields
client_data[var_list_num].isnull().sum().sort_values(ascending=False) / client_data.shape[0]

StatusCreatedYear    0.000003
GroupSize            0.000000
AttendeeGrossCost    0.000000
EventId              0.000000
StartYear            0.000000
dtype: float64

DROP nothing.

In [14]:
audit_num = client_data[var_list_num].describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99])
audit_num

,GroupSize,AttendeeGrossCost,EventId,StartYear,StatusCreatedYear
count,658446.000000,658446.000000,658446.000000,658446.000000,658444.000000
mean,1.013389,0.908920,20528.085304,2019.528696,2019.519561
std,0.200668,74.082132,3707.289450,1.472079,1.464403
min,0.000000,0.000000,0.000000,2017.000000,2017.000000
25%,1.000000,0.000000,17028.000000,2018.000000,2018.000000
50%,1.000000,0.000000,20051.000000,2019.000000,2019.000000
75%,1.000000,0.000000,24644.000000,2021.000000,2021.000000
90%,1.000000,0.000000,25662.000000,2022.000000,2022.000000
95%,1.000000,0.000000,26092.000000,2022.000000,2022.000000
99%,1.000000,0.000000,26412.000000,2022.000000,2022.000000


In [15]:
# Percentage of missing values in categoric fields
client_data[var_list_cat].isnull().sum().sort_values(ascending= False) / client_data.shape[0]

TicketType           0.747177
EventName            0.000000
EventType            0.000000
BookingReference     0.000000
AttendeeReference    0.000000
IsLeadAttendee       0.000000
BookingStatus        0.000000
AttendeeType         0.000000
dtype: float64

Drop TicketType due to high percentage of missing values.

### Drop unwanted fields

In [16]:
client_data.drop(['TicketType'], axis=1, inplace= True)     # Dropping ticketype

In [17]:
client_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658446 entries, 0 to 658445
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   EventName          658446 non-null  object        
 1   StartDate          658446 non-null  datetime64[ns]
 2   EventType          658446 non-null  object        
 3   BookingReference   658446 non-null  object        
 4   AttendeeReference  658446 non-null  object        
 5   GroupSize          658446 non-null  int64         
 6   IsLeadAttendee     658446 non-null  object        
 7   AttendeeGrossCost  658446 non-null  float64       
 8   EventId            658446 non-null  int64         
 9   BookingStatus      658446 non-null  object        
 10  AttendeeType       658446 non-null  object        
 11  StatusCreatedDate  658444 non-null  datetime64[ns]
 12  ClientId           658446 non-null  float64       
 13  StartYear          658446 non-null  int64   

In [18]:
client_data.head(3)

,EventName,StartDate,EventType,BookingReference,AttendeeReference,GroupSize,IsLeadAttendee,AttendeeGrossCost,EventId,BookingStatus,AttendeeType,StatusCreatedDate,ClientId,StartYear,StatusCreatedYear
0,Developing Reading and Writing Skills,2017-11-28,"Others, Group 1",B462687,A514297,1,True,0.0,14983,Attending,Attendee,2017-11-11 09:15:00,153.0,2017,2017.0
1,Teaching Grammar: Classroom choices,2017-12-13,"Others, Group 1",B462689,A514299,1,True,0.0,14960,Attending,Attendee,2017-11-11 09:18:00,153.0,2017,2017.0
2,Teaching Grammar: Classroom choices,2017-12-13,"Others, Group 1",B467150,A519135,1,True,0.0,14960,Attending,Attendee,2017-11-22 17:02:00,153.0,2017,2017.0


### Export datasets

In [19]:
# export
client_data.to_csv(os.path.join(processing_path,dataset_audited_fname) + dataset_fname_suffix,index = False)